In [1]:
# Cell 0: Imports & helper utilities
import pandas as pd
import numpy as np
from pathlib import Path
from math import ceil
from datetime import timedelta

pd.set_option('display.max_columns', 120)
pd.set_option('display.width', 120)

print("Ready — imports done.")

Ready — imports done.


In [3]:
# Cell 1: Load admissions.csv (or use sample if file not present) and expand to one row per day (day_index)
csv_path = Path("admissions.csv")

admissions = pd.read_csv(csv_path, low_memory=False,
                            parse_dates=['admittime','dischtime','deathtime','edregtime','edouttime'])
print("Loaded admissions.csv from disk. Rows:", len(admissions))

# Ensure datetime types
admissions['admittime'] = pd.to_datetime(admissions['admittime'], errors='coerce')
admissions['dischtime'] = pd.to_datetime(admissions['dischtime'], errors='coerce')

# If dischtime missing, fill with admittime (so at least one day is produced)
admissions['dischtime'] = admissions['dischtime'].fillna(admissions['admittime'])

# Function to expand a single admission row into day rows
def expand_admission_row(row):
    adm_date = row['admittime'].normalize().date()
    dis_date = row['dischtime'].normalize().date()
    n_days = (dis_date - adm_date).days + 1
    if n_days <= 0:
        n_days = 1
    rows = []
    for d in range(n_days):
        new = {
            'subject_id': row['subject_id'],
            'hadm_id': row['hadm_id'],
            'day_index': int(d),
            'admittime': row['admittime'],
            'dischtime': row['dischtime'],
            'admission_type': row.get('admission_type', np.nan),
            'admit_provider_id': row.get('admit_provider_id', np.nan),
            'admission_location': row.get('admission_location', np.nan),
            'discharge_location': row.get('discharge_location', np.nan),
            'insurance': row.get('insurance', np.nan),
            'language': row.get('language', np.nan),
            'marital_status': row.get('marital_status', np.nan),
            'race': row.get('race', np.nan),
            'edregtime': row.get('edregtime', pd.NaT),
            'edouttime': row.get('edouttime', pd.NaT),
            'hospital_expire_flag': row.get('hospital_expire_flag', np.nan)
        }
        rows.append(new)
    return rows

# Expand all admissions
expanded = []
for _, r in admissions.iterrows():
    expanded.extend(expand_admission_row(r))

merged_initial = pd.DataFrame(expanded)
# Convert types
merged_initial[['subject_id','hadm_id','day_index']] = merged_initial[['subject_id','hadm_id','day_index']].astype('Int64')

print("Expanded admissions -> rows:", merged_initial.shape[0])


Loaded admissions.csv from disk. Rows: 21987
Expanded admissions -> rows: 325800


In [4]:
merged_initial

,subject_id,hadm_id,day_index,admittime,dischtime,admission_type,admit_provider_id,admission_location,discharge_location,insurance,language,marital_status,race,edregtime,edouttime,hospital_expire_flag
0,10000935,26381316,0,2187-08-23 21:22:00,2187-08-27 15:35:00,EW EMER.,P52V4D,EMERGENCY ROOM,SKILLED NURSING FACILITY,Medicare,English,SINGLE,BLACK/AFRICAN AMERICAN,2187-08-23 14:37:00,2187-08-23 22:46:00,0
1,10000935,26381316,1,2187-08-23 21:22:00,2187-08-27 15:35:00,EW EMER.,P52V4D,EMERGENCY ROOM,SKILLED NURSING FACILITY,Medicare,English,SINGLE,BLACK/AFRICAN AMERICAN,2187-08-23 14:37:00,2187-08-23 22:46:00,0
2,10000935,26381316,2,2187-08-23 21:22:00,2187-08-27 15:35:00,EW EMER.,P52V4D,EMERGENCY ROOM,SKILLED NURSING FACILITY,Medicare,English,SINGLE,BLACK/AFRICAN AMERICAN,2187-08-23 14:37:00,2187-08-23 22:46:00,0
3,10000935,26381316,3,2187-08-23 21:22:00,2187-08-27 15:35:00,EW EMER.,P52V4D,EMERGENCY ROOM,SKILLED NURSING FACILITY,Medicare,English,SINGLE,BLACK/AFRICAN AMERICAN,2187-08-23 14:37:00,2187-08-23 22:46:00,0
4,10000935,26381316,4,2187-08-23 21:22:00,2187-08-27 15:35:00,EW EMER.,P52V4D,EMERGENCY ROOM,SKILLED NURSING FACILITY,Medicare,English,SINGLE,BLACK/AFRICAN AMERICAN,2187-08-23 14:37:00,2187-08-23 22:46:00,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
325795,19999828,25744818,6,2149-01-08 16:44:00,2149-01-18 17:00:00,EW EMER.,P13JMH,TRANSFER FROM HOSPITAL,HOME HEALTH CARE,Medicaid,English,SINGLE,WHITE,2149-01-08 09:11:00,2149-01-08 18:12:00,0
325796,19999828,25744818,7,2149-01-08 16:44:00,2149-01-18 17:00:00,EW EMER.,P13JMH,TRANSFER FROM HOSPITAL,HOME HEALTH CARE,Medicaid,English,SINGLE,WHITE,2149-01-08 09:11:00,2149-01-08 18:12:00,0
325797,19999828,25744818,8,2149-01-08 16:44:00,2149-01-18 17:00:00,EW EMER.,P13JMH,TRANSFER FROM HOSPITAL,HOME HEALTH CARE,Medicaid,English,SINGLE,WHITE,2149-01-08 09:11:00,2149-01-08 18:12:00,0
325798,19999828,25744818,9,2149-01-08 16:44:00,2149-01-18 17:00:00,EW EMER.,P13JMH,TRANSFER FROM HOSPITAL,HOME HEALTH CARE,Medicaid,English,SINGLE,WHITE,2149-01-08 09:11:00,2149-01-08 18:12:00,0
